In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
0,0,Leeton,AU,-34.57,146.40,46.40,100,75,3.36,broken clouds,0.0,0
1,1,Port Elizabeth,ZA,-33.92,25.57,48.20,66,0,8.05,clear sky,0.0,0
2,2,Nouadhibou,MR,20.93,-17.03,69.80,83,0,10.29,clear sky,0.0,0
3,3,Severnoye,RU,56.35,78.36,63.27,88,77,8.12,broken clouds,0.0,0
4,4,Busselton,AU,-33.65,115.33,48.99,74,0,16.98,clear sky,0.0,0


In [4]:
# Get the data types
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain(inches)           float64
Snow(inches)             int64
dtype: object

In [5]:
# Ask the customer to add a minimum & maximum temperature value and if the customer would like it to be raining and/or snowing. 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain(inches)"] == 0 if rain == "no" else city_data_df["Rain(inches)"] > 0) &
                                       (city_data_df["Snow(inches)"] == 0 if snow == "no" else city_data_df["Snow(inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain(inches),Snow(inches)
7,7,Matara,LK,5.95,80.54,79.07,86,100,8.55,overcast clouds,0.0,0
16,16,Adrar,MR,20.50,-10.07,89.37,36,34,9.13,scattered clouds,0.0,0
18,18,Kapaa,US,22.08,-159.32,84.20,70,75,17.22,broken clouds,0.0,0
27,27,Gizo,SB,-8.10,156.84,84.36,70,87,7.63,overcast clouds,0.0,0
30,30,Keningau,MY,5.34,116.16,78.80,83,20,6.93,few clouds,0.0,0
32,32,Pauini,BR,-7.71,-66.98,76.06,74,0,1.74,clear sky,0.0,0
36,36,Hithadhoo,MV,-0.60,73.08,83.48,79,0,15.05,clear sky,0.0,0
39,39,Bengkulu,ID,-3.80,102.27,78.33,79,100,2.77,overcast clouds,0.0,0
43,43,Kirakira,SB,-10.45,161.92,82.94,75,20,14.85,few clouds,0.0,0
45,45,Pacific Grove,US,36.62,-121.92,78.01,56,1,9.17,clear sky,0.0,0


In [7]:
preferred_cities_df.count()

City_ID                159
City                   159
Country                159
Lat                    159
Lng                    159
Max Temp               159
Humidity               159
Cloudiness             159
Wind Speed             159
Current Description    159
Rain(inches)           159
Snow(inches)           159
dtype: int64

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Matara,LK,79.07,overcast clouds,5.95,80.54,
16,Adrar,MR,89.37,scattered clouds,20.50,-10.07,
18,Kapaa,US,84.20,broken clouds,22.08,-159.32,
27,Gizo,SB,84.36,overcast clouds,-8.10,156.84,
30,Keningau,MY,78.80,few clouds,5.34,116.16,
32,Pauini,BR,76.06,clear sky,-7.71,-66.98,
36,Hithadhoo,MV,83.48,clear sky,-0.60,73.08,
39,Bengkulu,ID,78.33,overcast clouds,-3.80,102.27,
43,Kirakira,SB,82.94,few clouds,-10.45,161.92,
45,Pacific Grove,US,78.01,clear sky,36.62,-121.92,


In [9]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
 # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Matara,LK,79.07,overcast clouds,5.95,80.54,Amaloh Boutique Resort
16,Adrar,MR,89.37,scattered clouds,20.50,-10.07,
18,Kapaa,US,84.20,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
27,Gizo,SB,84.36,overcast clouds,-8.10,156.84,Imagination Island
30,Keningau,MY,78.80,few clouds,5.34,116.16,Sabindo Hotel
32,Pauini,BR,76.06,clear sky,-7.71,-66.98,Casa do Kiwliks
36,Hithadhoo,MV,83.48,clear sky,-0.60,73.08,Scoop Guest House
39,Bengkulu,ID,78.33,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
43,Kirakira,SB,82.94,few clouds,-10.45,161.92,SanBiz Lodge
45,Pacific Grove,US,78.01,clear sky,36.62,-121.92,Lovers Point Inn


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")